In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'
LAW2VEC_PATH = FOLDER / 'data/law2vec/Law2Vec.200d.txt'

# Math stuff
import numpy as np
from numpy.core.records import ndarray

# Data visualisation
import pandas as pd
from pandas import Series

# Vectorize algotrithm
from gensim.models import Word2Vec, KeyedVectors

# Support Vector Classifier Algorithm
from sklearn.svm import SVC

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
category = df['Subcategory']

## law2vec model

In [3]:
# Load a word2vec model stored in the C *text* format.
model = KeyedVectors.load_word2vec_format(LAW2VEC_PATH, binary=False)
l2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}


In [4]:
class MeanEmbeddingVectorizer(object):
    """Calculate the mean of each word"""
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(l2v_dict))])
        else:
            self.dim=0

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0) for words in X]
        )

In [5]:
# Use pipes to implement steps of fit and transform method
svm_l2v = Pipeline([
    # Add the words we want to mean
    ("law2vec vectorizer", MeanEmbeddingVectorizer(l2v_dict)),
    # Use SVC algorithm
    ("SVM", SVC())])

## Train SVM Model

In [6]:
# split data into test and train sets
# where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, category, random_state=42, test_size=0.3, shuffle=True)

In [7]:
# Fit SVM model with out train data
svm_l2v.fit(X_train, y_train)

Pipeline(steps=[('law2vec vectorizer',
                 <__main__.MeanEmbeddingVectorizer object at 0x7f4c476e4370>),
                ('SVM', SVC())])

In [8]:
# SVM prediction based on test data
prediction = svm_l2v.predict(X_test)
prediction

array([10, 14,  0,  2, 20,  0, 10, 10, 10, 10, 10, 22, 10,  0, 10,  2, 12,
       21, 21, 10, 14, 10,  0, 10, 10,  2, 10,  2, 14, 20, 10, 14, 14, 10,
        0, 20, 10, 10,  2,  9, 10, 20, 26, 14, 14,  2, 20, 20, 10, 14, 14,
       20, 14, 10, 28, 14,  9, 14, 12, 10, 14, 10,  0,  2, 10,  0, 10, 14,
       20, 10,  2, 10,  0,  0, 14,  9, 10, 14, 12, 21, 12, 14,  0, 10, 10,
        0,  0, 21, 21, 10,  2,  0,  0, 14, 10, 14,  9, 10,  0, 10,  2, 10,
       14, 21, 14, 10, 10, 20,  9, 21,  2,  0,  9,  0, 14, 14,  2, 20, 10,
       10,  9, 14,  9, 10,  2, 14, 14,  9, 14, 20, 10,  0, 10, 10, 10,  2,
       10, 10,  2, 14, 19,  2, 14, 20, 21, 10, 10, 10,  2, 10, 10, 14, 10,
       14, 22,  2, 10,  0, 14, 10, 10, 21, 14, 14, 22, 10, 14, 10, 10,  9,
        2,  2, 14, 22, 20, 10, 10, 10, 14,  0,  9, 14, 14, 10,  0,  0, 10,
       10, 14, 10, 10, 21, 12, 10, 26, 14, 12, 10, 21, 20, 21, 28,  0, 14,
        2, 14, 20, 28, 10, 19, 14, 10, 10, 14,  0,  9,  2,  9, 10,  0, 10,
       10, 20, 10, 21, 10

## Evaluation

In [9]:
model_evaluation_metrics(y_test, prediction, 'Category')

,Evaluation Metrics,Category
0,Accuracy,0.412811
1,Precission,0.165153
2,Recall,0.193903
3,F1 Score,0.165221
4,Mean Absolute Error,5.829181
5,Mean Squared Error,85.907473
